<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/evaluation/QuestionGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QuestionGeneration

This notebook walks through the process of generating a list of questions that could be asked about your data. This is useful for setting up an evaluation pipeline using the `FaithfulnessEvaluator` and `RelevancyEvaluator` evaluation tools.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [9]:
# %pip install llama-index-llms-openai

In [10]:
# !pip install llama-index

In [11]:
# attach to the same event-loop
import nest_asyncio

nest_asyncio.apply()

In [12]:
import logging
import sys
import pandas as pd

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [13]:
from llama_index.core.evaluation import DatasetGenerator, RelevancyEvaluator
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Response
# from llama_index.llms.openai import OpenAI

from jet.llm.ollama.base import initialize_ollama_settings, create_llm
initialize_ollama_settings()

_Settings(_llm=Ollama(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x10b7feff0>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x137eea020>, completion_to_prompt=<function default_completion_to_prompt at 0x137fd2ac0>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, base_url='http://localhost:11434', model='llama3.1', temperature=0.0, context_window=4096, request_timeout=300.0, prompt_key='prompt', json_mode=False, additional_kwargs={}, is_function_calling_model=True, keep_alive=None), _embed_model=OllamaEmbedding(model_name='nomic-embed-text', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x10b7feff0>, num_workers=None, base_url='http://localhost:11434', ollama_additional_kwargs={}), _callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x10b7feff0>, _tokenizer=None, _node_parser=SentenceSplitter

Download Data

In [14]:
# !mkdir -p 'data/paul_graham/'
# !wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

Load Data

In [15]:
data_dir = "/Users/jethroestrada/Desktop/External_Projects/AI/chatbot/open-webui/backend/crewAI/docs"
reader = SimpleDirectoryReader(data_dir)
documents = reader.load_data()

In [16]:
data_generator = DatasetGenerator.from_documents(documents)

/Users/jethroestrada/Desktop/External_Projects/AI/repo-libs/llama_index/llama-index-core/llama_index/core/evaluation/dataset_generation.py:200: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(


In [17]:
eval_questions = data_generator.generate_questions_from_nodes()

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/ch

KeyboardInterrupt: 

In [ ]:
eval_questions

['What were the two main things the author worked on before college?',
 'How did the author describe their early attempts at writing short stories?',
 'What type of computer did the author first work on for programming?',
 'What language did the author use for programming on the IBM 1401?',
 "What was the author's experience with programming on the 1401?",
 'What type of computer did the author eventually get for themselves?',
 "What was the author's initial plan for college?",
 'What made the author change their mind about studying philosophy?',
 "What sparked the author's interest in AI?",
 'What did the author realize about AI during their first year of grad school?',
 'What were the two art schools that the author applied to?',
 'How did the author end up at RISD?',
 'What was the purpose of the foundation classes at RISD?',
 'How did the author manage to pass the entrance exam for the Accademia di Belli Arti?',
 'What was the arrangement between the students and faculty at the Acc

In [ ]:
# gpt-4
gpt4 = create_llm(temperature=0, model="llama3.1")

In [ ]:
evaluator_gpt4 = RelevancyEvaluator(llm=gpt4)

In [ ]:
# create vector index
vector_index = VectorStoreIndex.from_documents(documents)

In [ ]:
# define jupyter display function
def display_eval_df(query: str, response: Response, eval_result: str) -> None:
    eval_df = pd.DataFrame(
        {
            "Query": query,
            "Response": str(response),
            "Source": (
                response.source_nodes[0].node.get_content()[:1000] + "..."
            ),
            "Evaluation Result": eval_result,
        },
        index=[0],
    )
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        },
        subset=["Response", "Source"]
    )
    display(eval_df)

In [ ]:
query_engine = vector_index.as_query_engine()
response_vector = query_engine.query(eval_questions[1])
eval_result = evaluator_gpt4.evaluate_response(
    query=eval_questions[1], response=response_vector
)

In [ ]:
display_eval_df(eval_questions[1], response_vector, eval_result)

,Query,Response,Source,Evaluation Result
0,How did the author describe their early attempts at writing short stories?,"The author described their early attempts at writing short stories as awful. They mentioned that their stories had hardly any plot and were mostly about characters with strong feelings, which they thought made the stories deep.","What I Worked On February 2021 Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep. The first programs I tried writing were on the IBM 1401 that our school district used for what was then called ""data processing."" This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights. The language we used was an early version of Fortran. You had to type programs on punch cards, then stack them in the...",YES
